In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the datetime module from the datetime library.
from datetime import datetime
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import API key
from config import g_key
# gmaps 
import gmaps
# Import the time module.
import time
# Import linear regression from the SciPy stats module.
from scipy.stats import linregress

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_database/WeatherPy_Challenge.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
0,0,Nikolskoye,RU,2019-11-16 01:31:09,59.70,30.79,39.20,80,90,8.95,overcast clouds,0.0,0.0
1,1,Yellowknife,CA,2019-11-16 01:31:09,62.45,-114.38,12.20,85,90,6.93,light snow,0.0,0.0
2,2,Beipiao,CN,2019-11-16 01:31:09,41.80,120.76,30.47,23,0,6.62,clear sky,0.0,0.0
3,3,Eyl,SO,2019-11-16 01:31:09,7.98,49.82,78.80,80,0,22.01,clear sky,0.0,0.0
4,4,Hobart,AU,2019-11-16 01:31:10,-42.88,147.33,60.01,35,40,24.16,scattered clouds,0.0,0.0


In [16]:
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))
rain_preferred = str(input("Do you want it to be raining? (yes/no) "))
snow_preferred = str(input("Do you want it to be snowing? (yes/no) "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75
Do you want it to be raining? (yes/no) no
Do you want it to be snowing? (yes/no) no


In [17]:
if rain_preferred == "yes" and snow_preferred == "yes":
    preferred_cities= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                       (city_data_df["Max Temp"]>= min_temp) & \
                                       (city_data_df["Rain inches (last 3 hrs)"]> 0.0) & \
                                       (city_data_df["Snow inches (last 3 hrs)"]> 0.0)]
elif rain_preferred == "yes" and snow_preferred == "no":
    preferred_cities= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                       (city_data_df["Max Temp"]>= min_temp) & \
                                       (city_data_df["Rain inches (last 3 hrs)"]> 0) & \
                                       (city_data_df["Snow inches (last 3 hrs)"] == 0)]
elif rain_preferred == "no" and snow_preferred == "yes":
    preferred_cities= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                       (city_data_df["Max Temp"]>= min_temp) & \
                                       (city_data_df["Rain inches (last 3 hrs)"] ==0) & \
                                       (city_data_df["Snow inches (last 3 hrs)"]>0)]
elif rain_preferred == "no" and snow_preferred == "no":
    preferred_cities= city_data_df.loc[(city_data_df["Max Temp"]<= max_temp) & \
                                       (city_data_df["Max Temp"]>= min_temp) & \
                                       (city_data_df["Rain inches (last 3 hrs)"] ==0.0) & \
                                       (city_data_df["Snow inches (last 3 hrs)"] ==0.0)]
# If an error is experienced, skip the city.
else:
    ""
preferred_cities

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Weather,Rain inches (last 3 hrs),Snow inches (last 3 hrs)
4,4,Hobart,AU,2019-11-16 01:31:10,-42.88,147.33,60.01,35,40,24.16,scattered clouds,0.0,0.0
8,8,Vila Franca Do Campo,PT,2019-11-16 01:31:11,37.72,-25.43,60.80,59,75,6.93,broken clouds,0.0,0.0
9,9,Comodoro Rivadavia,AR,2019-11-16 01:31:11,-45.87,-67.48,62.60,25,100,26.40,overcast clouds,0.0,0.0
14,14,Jamestown,AU,2019-11-16 01:31:12,-33.21,138.60,72.93,19,0,8.16,clear sky,0.0,0.0
15,15,Ribeira Grande,PT,2019-11-16 01:31:13,38.52,-28.70,58.03,66,51,1.92,broken clouds,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
529,529,Lasa,CY,2019-11-16 01:33:14,34.92,32.53,68.00,77,14,3.36,few clouds,0.0,0.0
534,534,Aldergrove,CA,2019-11-16 01:33:15,49.06,-122.49,54.00,87,20,8.05,few clouds,0.0,0.0
535,535,Puerto Palomas,US,2019-11-16 01:33:15,31.78,-107.63,53.60,34,1,4.70,clear sky,0.0,0.0
536,536,Xucheng,CN,2019-11-16 01:33:16,33.00,118.51,71.87,24,0,4.50,clear sky,0.0,0.0


In [18]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities[["City", "Country", "Max Temp", "Current Weather", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Hobart,AU,60.01,scattered clouds,-42.88,147.33,
8,Vila Franca Do Campo,PT,60.80,broken clouds,37.72,-25.43,
9,Comodoro Rivadavia,AR,62.60,overcast clouds,-45.87,-67.48,
14,Jamestown,AU,72.93,clear sky,-33.21,138.60,
15,Ribeira Grande,PT,58.03,broken clouds,38.52,-28.70,


In [19]:
# Set parameters to search for a hotel.
params = {"radius": 8000, "type": "lodging", "key": g_key}

In [20]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to the params dictionary as values to the location key.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
# Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels['results'][0]['name']
        
    except (IndexError, NameError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [21]:
# Create the output file (CSV).
output_data_file = "weather_database/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [22]:
hotel_df.head()

,City,Country,Max Temp,Current Weather,Lat,Lng,Hotel Name
4,Hobart,AU,60.01,scattered clouds,-42.88,147.33,St Ives Apartments
8,Vila Franca Do Campo,PT,60.80,broken clouds,37.72,-25.43,Pestana Bahia Praia
9,Comodoro Rivadavia,AR,62.60,overcast clouds,-45.87,-67.48,Austral Hotel
14,Jamestown,AU,72.93,clear sky,-33.21,138.60,Jamestown Motel Terraces
15,Ribeira Grande,PT,58.03,broken clouds,38.52,-28.70,Hotel do Canal | Horta - Faial


In [23]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key = g_key)

In [24]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Weather} </dd>and<dd>{Max Temp} °F</dd>
</dl>
"""

# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))